# Comparing similar suburbs in the vicinity Philadelphia, PA

## Introduction

### When I moved to the town I now live in, our selection criteria was good schools. Over time, I have grown to love the quaint shops and independent restuarants. Now that the kids are grown and we are downsizing, I'd like to find a similar neighborhood with these amenities and low property taxes since I no longer care about the quality of the schools.

## Data

### To accomplish this I will need to gather data on neighborhoods around Philadelphia, property tax data, and venue information. The data will need to be joined by zipcode and/or municipality.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Download Neighborhood and zipcode data for the Philadelphia metro area

#### This data can be found on this website: https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-alpha/Philadelphia%20(PA-NJ)1.html
#### This data includes the zipcode and minicipality, telephone area code, county, state, and some extraneous information that I will not need.

In [2]:
base_site = 'https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Philadelphia%20(PA-NJ)1.html'

r = requests.get(base_site)
r.status_code

DF_list = pd.read_html(r.text)

Philly_DF=DF_list[0]
Philly_DF.head()

,0,1,2,3,4,5
0,08001 Alloway,856,Salem County,New Jersey - NJ,"Philadelphia, PA-NJ (6160)",SMSA
1,08001 Paradise Lakes,856,Salem County,New Jersey - NJ,"Philadelphia, PA-NJ (6160)",SMSA
2,08002 Cherry Hill,856,Camden County,New Jersey - NJ,"Philadelphia, PA-NJ (6160)",SMSA
3,08002 Cherry Hill Township,856,Camden County,New Jersey - NJ,"Philadelphia, PA-NJ (6160)",SMSA
4,08002 Ellisburg,856,Camden County,New Jersey - NJ,"Philadelphia, PA-NJ (6160)",SMSA


#### Retain only Delaware County and Montgomery Counties in PA

In [3]:
PA_DF = Philly_DF[Philly_DF[3].str.contains('PA')]
DelCo_DF=PA_DF[PA_DF[2].str.contains('Delaware')]
MontCo_DF=PA_DF[PA_DF[2].str.contains('Montgomery')]
DelCoMontCo_DF=pd.concat([DelCo_DF, MontCo_DF], axis=0)
DelCoMontCo_DF.head()

,0,1,2,3,4,5
389,19008 Broomall,610/484,Delaware County,Pennsylvania - PA,"Philadelphia, PA-NJ (6160)",SMSA
390,19008 Lawrence Park,610/484,Delaware County,Pennsylvania - PA,"Philadelphia, PA-NJ (6160)",SMSA
391,19008 Marple Township,610/484,Delaware County,Pennsylvania - PA,"Philadelphia, PA-NJ (6160)",SMSA
392,19008 Radnor,610/484,Delaware County,Pennsylvania - PA,"Philadelphia, PA-NJ (6160)",SMSA
394,19010 Bryn Mawr,610/484,Delaware County,Pennsylvania - PA,"Philadelphia, PA-NJ (6160)",SMSA


#### Drop Columns 1, 2, 3, 4, and 5 since they are not useful, split the zip code and municipality column into separate columns and rename them.

In [4]:
DelCoMontCo_DF.drop([1,2,3,4,5], axis=1,inplace=True)
DelCoMontCo_DF.columns = ['Zip_Muni']
DelCoMontCo_DF=pd.DataFrame(DelCoMontCo_DF.Zip_Muni.str.split(' ',1).tolist(), columns = ['Zip','MUNICIPALITY'])
DelCoMontCo_DF['Zip'] = DelCoMontCo_DF['Zip'].astype('int64')
DelCoMontCo_DF['MUNICIPALITY'] = DelCoMontCo_DF['MUNICIPALITY'].str.upper() 
DelCoMontCo_DF.head()

,Zip,MUNICIPALITY
0,19008,BROOMALL
1,19008,LAWRENCE PARK
2,19008,MARPLE TOWNSHIP
3,19008,RADNOR
4,19010,BRYN MAWR


### 2. Download Property tax information for Delaware County, PA

#### This data can be found on this website: https://www.delcopa.gov/treasurer/propertytaxes.html as a PDF. I exported the PDF to Excel, trimmed out extranous information, and renamed teh columns.

In [5]:
DelCoTax_DF=pd.read_excel('TaxRateDelCo.xlsx', index_col=0) 
DelCoTax_DF.head()

,TotalTaxRate
MUNICIPALITY,
ALDAN,60.5610
ASTON,40.8413
BETHEL,40.7660
CHADDS FORD,32.0530
BROOKHAVEN,39.9913


### 3. Download Property tax information for Montgomery County, PA

#### This data can be found on this website: https://www.montcopa.org/622/County-Municipality-Millage-Rates

In [6]:
base_site = 'https://www.montcopa.org/622/County-Municipality-Millage-Rates'

r = requests.get(base_site)
r.status_code

DF_list = pd.read_html(r.text)

MontCoTax_DF=DF_list[0]
MontCoTax_DF.columns = MontCoTax_DF.iloc[0]
MontCoTax_DF.drop([0], axis=0,inplace=True)
MontCoTax_DF.head()

,Code,Municipality,Municipality Bill Date,School Bill Date,2020 County Millage,2020 MCCC Millage,2020 Municipality Millage,2019-2020 School Millage,Total Millage
1,01,Ambler,02/28/20,07/01/19,3.459,0.39,7.86,20.59,32.299
2,02,Bridgeport,02/28/20,07/01/19,3.459,0.39,12.23,20.34,36.419
3,03,Bryn Athyn,02/01/20,07/01/19,3.459,0.39,10.876,0,14.725
4,04,Collegeville,03/01/20,07/01/19,3.459,0.39,5.2,33.72,42.769
5,05,Conshohocken,02/01/20,07/01/19,3.459,0.39,4.5,22.772,31.121


#### I'm really only interested in the municipality and total millage.

In [7]:
MontCoTax_DF=MontCoTax_DF[['Municipality','Total Millage']]
MontCoTax_DF.columns = ['MUNICIPALITY','TotalTaxRate']
MontCoTax_DF['MUNICIPALITY']=MontCoTax_DF['MUNICIPALITY'].str.upper()
MontCoTax_DF.head()

C:\Users\Marylin Huff\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,MUNICIPALITY,TotalTaxRate
1,AMBLER,32.299
2,BRIDGEPORT,36.419
3,BRYN ATHYN,14.725
4,COLLEGEVILLE,42.769
5,CONSHOHOCKEN,31.121


### 4. Link this information into a single DataFrame

In [8]:
DelCoMontCoTax_DF=pd.concat([DelCoTax_DF, MontCoTax_DF], axis=0)
DelCoMontCo_DF=pd.merge(DelCoMontCo_DF, DelCoMontCoTax_DF, on='MUNICIPALITY')
DelCoMontCo_DF.head()

,Zip,MUNICIPALITY,TotalTaxRate
0,19063,UPPER PROVIDENCE,32.5267
1,19064,SPRINGFIELD,42.0002
2,18041,EAST GREENVILLE,35.3268
3,18054,GREEN LANE,31.0768
4,18073,PENNSBURG,36.3018


### 5. Coordinates can be found based on US Zip Codes on this website: 
#### https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=PA
### I downloaded the data as a excel file and trimmed out the extraneous information.

In [9]:
ZipLngLat_DF=pd.read_excel('us-zip-code-latitude-and-longitude.xls') 
ZipLngLat_DF.head()

,Zip,Latitude,Longitude
0,17932,40.649109,-76.503339
1,18947,40.426391,-75.118420
2,15278,40.434436,-80.024817
3,15482,40.062849,-79.763380
4,15227,40.377869,-79.975160


In [10]:
DelCoMontCo_DF=DelCoMontCo_DF.merge(ZipLngLat_DF, how='left', on='Zip')
DelCoMontCo_DF.head()

,Zip,MUNICIPALITY,TotalTaxRate,Latitude,Longitude
0,19063,UPPER PROVIDENCE,32.5267,39.916078,-75.40251
1,19064,SPRINGFIELD,42.0002,39.929511,-75.33562
2,18041,EAST GREENVILLE,35.3268,40.410224,-75.50845
3,18054,GREEN LANE,31.0768,40.343587,-75.44613
4,18073,PENNSBURG,36.3018,40.388263,-75.48659


### 6. Create a map of the region

In [11]:
# create map using latitude and longitude values
latitude=DelCoMontCo_DF['Latitude'].mean()
longitude=DelCoMontCo_DF['Longitude'].mean()
map_burbs = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# add markers to map
for lat, lng, label in zip(DelCoMontCo_DF['Latitude'], DelCoMontCo_DF['Longitude'], DelCoMontCo_DF['MUNICIPALITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_burbs)  
    
map_burbs

### 7. Access Foursquare

In [12]:
CLIENT_ID = '3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD' # your Foursquare ID
CLIENT_SECRET = 'QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0' # your Foursquare Secret
VERSION = '20200625'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD
CLIENT_SECRET:QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0


### Find venues within 500 meters of each Municipality

In [13]:
LIMIT=25
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['MUNICIPALITY', 
                  'Muni Latitude', 
                  'Muni Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
burb_venues = getNearbyVenues(names=DelCoMontCo_DF['MUNICIPALITY'], latitudes=DelCoMontCo_DF['Latitude'], longitudes=DelCoMontCo_DF['Longitude'])
burb_venues.head()

UPPER PROVIDENCE
SPRINGFIELD
EAST GREENVILLE
GREEN LANE
PENNSBURG
RED HILL
RED HILL
FRANCONIA
SALFORD
SOUDERTON
TELFORD
ABINGTON
AMBLER
BRYN ATHYN
CHELTENHAM
UPPER DUBLIN
HORSHAM
JENKINTOWN
ROCKLEDGE
NARBERTH
EAST NORRITON
EAST NORRITON
NORRISTOWN
NORRISTOWN
NORRISTOWN
NORRISTOWN
NORRISTOWN
NORRISTOWN
NORRISTOWN
NORRISTOWN
BRIDGEPORT
COLLEGEVILLE
COLLEGEVILLE
TRAPPE
CONSHOHOCKEN
CONSHOHOCKEN
WEST CONSHOHOCKEN
NORTH WALES
NORTH WALES
NORTH WALES
NORTH WALES
HATFIELD
LANSDALE
UPPER GWYNEDD
POTTSTOWN
LIMERICK
ROYERSFORD
SCHWENKSVILLE
SKIPPACK
WORCESTER
DOUGLASS


,MUNICIPALITY,Muni Latitude,Muni Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,UPPER PROVIDENCE,39.916078,-75.40251,G-Fit Women,39.916672,-75.401550,Gym / Fitness Center
1,UPPER PROVIDENCE,39.916078,-75.40251,VFW Post 3460,39.915794,-75.401703,Nightlife Spot
2,SPRINGFIELD,39.929511,-75.33562,Wyndmoor Brewery,39.930249,-75.336215,Brewery
3,SPRINGFIELD,39.929511,-75.33562,Springfield High School Track,39.928391,-75.337006,Athletics & Sports
4,SPRINGFIELD,39.929511,-75.33562,Springfield HS Track,39.928738,-75.336740,Track


### Analyze Venues in each Municipality

In [15]:
# one hot encoding
burb_onehot = pd.get_dummies(burb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
burb_onehot['MUNICIPALITY'] = burb_venues['MUNICIPALITY'] 

# move municipality column to the first column
fixed_columns = [burb_onehot.columns[-1]] + list(burb_onehot.columns[:-1])
burb_onehot = burb_onehot[fixed_columns]
burb_grouped = burb_onehot.groupby('MUNICIPALITY').mean().reset_index()
burb_grouped.head()

,MUNICIPALITY,American Restaurant,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar,Doctor's Office,Donut Shop,Event Service,Fast Food Restaurant,Food,Gym / Fitness Center,Home Service,Italian Restaurant,Light Rail Station,Motorcycle Shop,Nightlife Spot,Park,Pharmacy,Playground,Restaurant,Sandwich Place,Shopping Mall,Tourist Information Center,Track
0,ABINGTON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AMBLER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,BRIDGEPORT,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BRYN ATHYN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CHELTENHAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Create a dataframe of the most common venues in each Municipality

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['MUNICIPALITY']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
munis_venues_sorted = pd.DataFrame(columns=columns)
munis_venues_sorted['MUNICIPALITY'] = burb_grouped['MUNICIPALITY']

for ind in np.arange(burb_grouped.shape[0]):
    munis_venues_sorted.iloc[ind, 1:] = return_most_common_venues(burb_grouped.iloc[ind, :], num_top_venues)

munis_venues_sorted.head()

,MUNICIPALITY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABINGTON,Shopping Mall,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
1,AMBLER,Playground,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
2,BRIDGEPORT,Deli / Bodega,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Dive Bar,Doctor's Office
3,BRYN ATHYN,Pharmacy,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
4,CHELTENHAM,Park,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar


### 8. Cluster the Municipalities

In [18]:
# set number of clusters
kclusters = 8

burb_grouped_clustering = burb_grouped.drop('MUNICIPALITY', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(burb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 6, 2, 5, 4, 0, 2, 2, 2, 2])

### Creating a new dataframe with cluster and tax information

In [19]:
# add clustering labels
munis_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

burb_merged = DelCoMontCo_DF

# merge burb_grouped with DelCoMontCo_DF to add latitude/longitude and tax info for each neighborhood
burb_merged = burb_merged.join(munis_venues_sorted.set_index('MUNICIPALITY'), on='MUNICIPALITY')
burb_merged.dropna(axis=0, how='any', inplace=True)
burb_merged['Cluster Labels'] = burb_merged['Cluster Labels'].astype('int64')
burb_merged # check the last columns!

,Zip,MUNICIPALITY,TotalTaxRate,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,19063,UPPER PROVIDENCE,32.5267,39.916078,-75.402510,2,Gym / Fitness Center,Nightlife Spot,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
1,19064,SPRINGFIELD,42.0002,39.929511,-75.335620,2,Track,Athletics & Sports,Brewery,Light Rail Station,Food,Bar,Big Box Store,Business Service,Deli / Bodega,Dive Bar
2,18041,EAST GREENVILLE,35.3268,40.410224,-75.508450,2,Sandwich Place,Big Box Store,Track,Food,Athletics & Sports,Bar,Brewery,Business Service,Deli / Bodega,Dive Bar
5,18073,RED HILL,31.7768,40.388263,-75.486590,4,Park,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
6,18076,RED HILL,31.7768,40.375182,-75.484310,4,Park,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
11,19001,ABINGTON,40.47,40.127907,-75.126950,7,Shopping Mall,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
12,19002,AMBLER,32.299,40.169608,-75.211580,6,Playground,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
13,19009,BRYN ATHYN,14.725,40.139539,-75.067160,5,Pharmacy,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
14,19012,CHELTENHAM,61.6958,40.060304,-75.104390,4,Park,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
15,19034,UPPER DUBLIN,44.2612,40.134770,-75.203740,2,Business Service,Doctor's Office,Tourist Information Center,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Deli / Bodega


### Create a map

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(burb_merged['Latitude'], burb_merged['Longitude'], burb_merged['MUNICIPALITY'], burb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### List Municipalities in Cluster #2 in order of Tax Rate

In [21]:
Cluster2 = burb_merged.loc[burb_merged["Cluster Labels"] == 2]
Cluster2.sort_values(by=["TotalTaxRate"], axis=0, ascending=True, inplace=True)
Cluster2

C:\Users\Marylin Huff\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Zip,MUNICIPALITY,TotalTaxRate,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,19428,WEST CONSHOHOCKEN,25.369,40.077736,-75.305090,2,Bar,Deli / Bodega,Dive Bar,Track,Food,Athletics & Sports,Big Box Store,Brewery,Business Service,Doctor's Office
35,19429,CONSHOHOCKEN,31.121,40.211896,-75.355860,2,Bar,Deli / Bodega,Dive Bar,Track,Food,Athletics & Sports,Big Box Store,Brewery,Business Service,Doctor's Office
34,19428,CONSHOHOCKEN,31.121,40.077736,-75.305090,2,Bar,Deli / Bodega,Dive Bar,Track,Food,Athletics & Sports,Big Box Store,Brewery,Business Service,Doctor's Office
0,19063,UPPER PROVIDENCE,32.5267,39.916078,-75.402510,2,Gym / Fitness Center,Nightlife Spot,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
16,19044,HORSHAM,34.513,40.184758,-75.143860,2,Restaurant,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
41,19440,HATFIELD,35.1657,40.278294,-75.294790,2,American Restaurant,Bar,Italian Restaurant,Food,Athletics & Sports,Big Box Store,Brewery,Business Service,Deli / Bodega,Dive Bar
2,18041,EAST GREENVILLE,35.3268,40.410224,-75.508450,2,Sandwich Place,Big Box Store,Track,Food,Athletics & Sports,Bar,Brewery,Business Service,Deli / Bodega,Dive Bar
30,19405,BRIDGEPORT,36.419,40.103208,-75.340500,2,Deli / Bodega,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Business Service,Dive Bar,Doctor's Office
1,19064,SPRINGFIELD,42.0002,39.929511,-75.335620,2,Track,Athletics & Sports,Brewery,Light Rail Station,Food,Bar,Big Box Store,Business Service,Deli / Bodega,Dive Bar
15,19034,UPPER DUBLIN,44.2612,40.134770,-75.203740,2,Business Service,Doctor's Office,Tourist Information Center,Track,Food,Athletics & Sports,Bar,Big Box Store,Brewery,Deli / Bodega
